# Build Knowledge Graph

In [1]:
entity={}
with open("kg-data-master/umls/entities.txt","r") as f:
    for line in f:
        line=line.split()
        entity[line[0]]=line[1]
print(entity)

relation={}
with open("kg-data-master/umls/relations.txt","r") as f:
    for line in f:
        line=line.split()
        relation[line[0]]=line[1]
print(relation)

entity_type={}
with open("kg-data-master/umls/entity_category.txt","r") as f:
    for line in f:
        line=line.split()
        entity_type[line[0]]=line[1]
print(entity_type)

{'0': 'Acquired_Abnormality', '1': 'Activity', '2': 'Age_Group', '3': 'Alga', '4': 'Amino_Acid,_Peptide,_or_Protein', '5': 'Amino_Acid_Sequence', '6': 'Amphibian', '7': 'Anatomical_Abnormality', '8': 'Anatomical_Structure', '9': 'Animal', '10': 'Antibiotic', '11': 'Archaeon', '12': 'Bacterium', '13': 'Behavior', '14': 'Biologic_Function', '15': 'Biologically_Active_Substance', '16': 'Biomedical_Occupation_or_Discipline', '17': 'Biomedical_or_Dental_Material', '18': 'Bird', '19': 'Body_Location_or_Region', '20': 'Body_Part,_Organ,_or_Organ_Component', '21': 'Body_Space_or_Junction', '22': 'Body_Substance', '23': 'Body_System', '24': 'Carbohydrate', '25': 'Carbohydrate_Sequence', '26': 'Cell', '27': 'Cell_Component', '28': 'Cell_Function', '29': 'Cell_or_Molecular_Dysfunction', '30': 'Chemical', '31': 'Chemical_Viewed_Functionally', '32': 'Chemical_Viewed_Structurally', '33': 'Classification', '34': 'Clinical_Attribute', '35': 'Clinical_Drug', '36': 'Conceptual_Entity', '37': 'Congenital

In [2]:
class Node1:
    def __init__(self,etype):
        self.etype=etype
        self.p=[]
        self.bp=[]
        
class Node2:
    def __init__(self,obj,pred):
        self.obj=obj
        self.pred=pred
        
class KG:
    def __init__(self):
        self.d={}
        
    def insert(self,sub,pred,obj,subtype,objtype):
        if sub not in self.d:
            self.d[sub]=Node1(subtype)
        
        if obj not in self.d:
            self.d[obj]=Node1(objtype)
            
        self.d[sub].p.append(Node2(obj,pred))
        self.d[obj].bp.append(Node2(sub,pred))
        
K=KG()
with open("kg-data-master/umls/triples.txt","r") as f:
    for line in f:
        line=line.split()
        K.insert(entity[line[1]],relation[line[0]],entity[line[2]],entity_type[line[1]],entity_type[line[2]])


In [3]:
entity=input('enter the entity to form QA pairs: ')
objects=K.d[entity].p
print(objects[0].pred)
tuples1=[]
for i in range(len(objects)):
    tuples1.append([entity,K.d[entity].etype,objects[i].pred,objects[i].obj,K.d[objects[i].obj].etype])
print(tuples1[1])

enter the entity to form QA pairs: Bacterium
causes
['Bacterium', 'Living_Beings', 'causes', 'Disease_or_Syndrome', 'Disorders']


# Unique Forward Relation

In [4]:
di={}
for i in tuples1:
    if i[2] not in di:
        di[i[2]]=1
    else:
        di[i[2]]+=1

unique=[]
for i in tuples1:
    if di[i[2]]!=None:
        unique.append(i)
        di[i[2]]=None
print(unique)

[['Bacterium', 'Living_Beings', 'causes', 'Cell_or_Molecular_Dysfunction', 'Disorders'], ['Bacterium', 'Living_Beings', 'interacts_with', 'Amphibian', 'Living_Beings'], ['Bacterium', 'Living_Beings', 'isa', 'Entity', 'Objects'], ['Bacterium', 'Living_Beings', 'issue_in', 'Biomedical_Occupation_or_Discipline', 'Occupations'], ['Bacterium', 'Living_Beings', 'location_of', 'Biologically_Active_Substance', 'Chemicals_&_Drugs']]


In [5]:
keywords=[]
answers=[]
for i in unique:
    keywords.append(' '.join([i[0],i[2],i[4]]))
    answers.append(i[3])

# Unique Reverse Relation

In [6]:
objects=K.d[entity].bp
print(objects[0].pred)
tuples2=[]
for i in range(len(objects)):
    tuples2.append([objects[i].obj,K.d[objects[i].obj].etype,objects[i].pred,entity,K.d[entity].etype])
print(tuples2[1])

affects
['Anatomical_Abnormality', 'Disorders', 'affects', 'Bacterium', 'Living_Beings']


In [7]:
di={}
for i in tuples2:
    if i[2] not in di:
        di[i[2]]=1
    else:
        di[i[2]]+=1

unique=[]
for i in tuples2:
    if di[i[2]]!=None:
        unique.append(i)
        di[i[2]]=None
print(unique)

[['Acquired_Abnormality', 'Disorders', 'affects', 'Bacterium', 'Living_Beings'], ['Immunologic_Factor', 'Chemicals_&_Drugs', 'indicates', 'Bacterium', 'Living_Beings'], ['Alga', 'Living_Beings', 'interacts_with', 'Bacterium', 'Living_Beings'], ['Acquired_Abnormality', 'Disorders', 'location_of', 'Bacterium', 'Living_Beings'], ['Acquired_Abnormality', 'Disorders', 'part_of', 'Bacterium', 'Living_Beings'], ['Biologic_Function', 'Phenomena', 'process_of', 'Bacterium', 'Living_Beings'], ['Clinical_Attribute', 'Physiology', 'property_of', 'Bacterium', 'Living_Beings']]


In [8]:
for i in unique:
    keywords.append(' '.join([i[0],i[2],i[4]]))
    answers.append(i[3])

In [9]:
import numpy as np
keywords=np.array(keywords,dtype='U74')

In [10]:
keywords

array(['Bacterium causes Disorders',
       'Bacterium interacts_with Living_Beings', 'Bacterium isa Objects',
       'Bacterium issue_in Occupations',
       'Bacterium location_of Chemicals_&_Drugs',
       'Acquired_Abnormality affects Living_Beings',
       'Immunologic_Factor indicates Living_Beings',
       'Alga interacts_with Living_Beings',
       'Acquired_Abnormality location_of Living_Beings',
       'Acquired_Abnormality part_of Living_Beings',
       'Biologic_Function process_of Living_Beings',
       'Clinical_Attribute property_of Living_Beings'], dtype='<U74')

# Data preprocessing

In [11]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs
 
# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

    # load dataset
filename = 'dataset.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'k2q.pkl')
# spot check
for i in range(100):
    print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: k2q.pkl
[what movies did sally field win an oscar for] => [movies field win oscar]
[who played mulder in the x files] => [played mulder x files]
[what is the name of dawn frenchs first novel] => [name dawn french first novel]
[what song is monifah famous for] => [song famous]
[what illness does michael j fox have] => [illness michael j fox]
[what are some of the awards taylor swift has won] => [awards taylor swift]
[who is ben roethlisberger parents] => [ben roethlisberger parents]
[what is modern egyptian language] => [modern egyptian language]
[who plays the voice of lois griffin] => [plays voice lois griffin]
[who did lenny kravitz marry] => [lenny kravitz marry]
[what money system does australia use] => [money system australia use]
[what to do in roatan bay] => [roatan bay]
[who plays jacob black in the twilight movies] => [plays jacob black twilight movies]
[what to see outside of paris] => [see outside paris]
[who does christopher walken play in batman returns] => [christo

In [12]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
 
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

raw_dataset = load_clean_sentences('k2q.pkl')
 
# reduce dataset size
n_sentences = 10000
dataset = raw_dataset[:, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:2100], dataset[2100:]
# save
save_clean_data(dataset, 'k2q-both.pkl')
save_clean_data(train, 'k2q-train.pkl')
save_clean_data(test, 'k2q-test.pkl')

Saved: k2q-both.pkl
Saved: k2q-train.pkl
Saved: k2q-test.pkl


In [13]:
import numpy as np
voc=np.array(list(dataset[:,1])+list(keywords),dtype='U74')

# Training RNN model

In [14]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
 
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y
 
# define RNN model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model
 
# load datasets
dataset = load_clean_sentences('k2q-both.pkl')
train = load_clean_sentences('k2q-train.pkl')
test = load_clean_sentences('k2q-test.pkl')
 
# prepare question tokenizer
q_tokenizer = create_tokenizer(dataset[:, 0])
q_vocab_size = len(q_tokenizer.word_index) + 1
q_length = max_length(dataset[:, 0])
print('Question Vocabulary Size: %d' % q_vocab_size)
print('Question Max Length : %d' % (q_length))
# prepare keyword tokenizer
k_tokenizer = create_tokenizer(voc)
k_vocab_size = len(k_tokenizer.word_index) + 1
k_length = max_length(voc)
print('Keyword Vocabulary Size: %d' % k_vocab_size)
print('Keyword Max Length: %d' % (k_length))
 
# prepare training data
trainX = encode_sequences(k_tokenizer, k_length, train[:, 1])
trainY = encode_sequences(q_tokenizer, q_length, train[:, 0])
z=trainY[:]

trainY = encode_output(trainY, q_vocab_size)

testX = encode_sequences(k_tokenizer, k_length, test[:, 1])
testY = encode_sequences(q_tokenizer, q_length, test[:, 0])

testY = encode_output(testY, q_vocab_size)

 
# define model
model = define_model(k_vocab_size, q_vocab_size, k_length, q_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

print(model.summary())

filename = 'modelcopy.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Using TensorFlow backend.


Question Vocabulary Size: 2690
Question Max Length : 13
Keyword Vocabulary Size: 2510
Keyword Max Length: 7
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 256)            642560    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 13, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 13, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 13, 2690)          691330    
Total params: 2,384,514
Trainable params: 2,384,514
Non-trainable params: 0
_________________________________________________________________
None
Train on 2100 sampl

In [36]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
 
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X
 
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)
 
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    k=list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        question = predict_sequence(model, tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        question = question.split()
        question = ' '.join(question[:3])+' '+raw_src
        
        #if i < 10:
        print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, question))
        
        actual.append(raw_target.split())
        predicted.append(question.split())
        k.append(raw_src.split())
    # calculate BLEU score
    #print(actual)
    #print('k=',k)
    return (actual,k,predicted)
    
def evaluate_model2(model, tokenizer, sources,keywords):
    #actual, predicted = list(), list()
    for i, source in enumerate(sources):
        
        source = source.reshape((1, source.shape[0]))
        question = predict_sequence(model, tokenizer, source)
        question = question.split()
        question = ' '.join(question[:3])+' '+keywords[i]
        #question= list(set(question.split()+keywords[i].split()))
        #question = list(set((question+keywords[i]).split()))
        #question = ' '.join(question)
        print('predicted question=[%s]  answer=[%s]' % (question+'?',answers[i]))
        

    # load datasets
'''dataset = load_clean_sentences('k2q-both.pkl')
train = load_clean_sentences('k2q-train.pkl')
test = load_clean_sentences('k2q-test.pkl')'''
# prepare question tokenizer
'''q_tokenizer = create_tokenizer(dataset[:, 0])
q_vocab_size = len(q_tokenizer.word_index) + 1
q_length = max_length(dataset[:, 0])
# prepare keyword tokenizer
k_tokenizer = create_tokenizer(dataset[:, 1])
k_vocab_size = len(k_tokenizer.word_index) + 1
k_length = max_length(dataset[:, 1])'''
# prepare data
trainX = encode_sequences(k_tokenizer, k_length, test[:, 1])
testX = encode_sequences(k_tokenizer, k_length, keywords)

#trainX.reshape((2100,13,1))
#trainX=array(trainX,dtype=float)
#testX.reshape((105,13,1))
#testX=array(trainX,dtype=float)
 
# load model
model = load_model('modelcopy.h5')
# test on some training sequences
#print('train')
#evaluate_model2(model, q_tokenizer, trainX)
# test on some test sequences
print('test:')
actual,k,predicted=evaluate_model(model, q_tokenizer, trainX,test)

#keywords=keywords.tolist()
print('\nQuestion Answer pairs:')
evaluate_model2(model, q_tokenizer, testX,keywords)

test:
src=[current queen australia], target=[who is the current queen of australia], predicted=[what is the current queen australia]
src=[fairfield], target=[what to do at fairfield ca], predicted=[what what is fairfield]
src=[year spudd webb dunk contest], target=[what year did spudd webb won the dunk contest], predicted=[what did did year spudd webb dunk contest]
src=[played maxie jones general hospital], target=[who played maxie jones on general hospital], predicted=[what is the played maxie jones general hospital]
src=[major languages italy], target=[what are the major languages of italy], predicted=[what is the major languages italy]
src=[language speak indonesia], target=[what language do they speak in indonesia], predicted=[what is the language speak indonesia]
src=[st francis patron saint], target=[what is st francis patron saint of], predicted=[what is the st francis patron saint]
src=[kids phx az], target=[what to do with kids in phx az], predicted=[what is the kids phx az]
s

src=[president jfk died], target=[who was the president after jfk died], predicted=[what did the president jfk died]
src=[college charles richard drew go], target=[what college did charles richard drew go to], predicted=[what did the college charles richard drew go]
src=[airport daytona beach florida], target=[what airport is close to daytona beach florida], predicted=[what did the airport daytona beach florida]
src=[chad michael murray play], target=[what does chad michael murray play in], predicted=[what did the chad michael murray play]
src=[currency use costa rica], target=[what currency do you use in costa rica], predicted=[what did the currency use costa rica]
src=[kansas city chiefs play], target=[where do the kansas city chiefs play], predicted=[what did the kansas city chiefs play]
src=[discovery sir isaac make force], target=[what discovery did sir isaac newton make about force], predicted=[what did the discovery sir isaac make force]
src=[al arabic names], target=[what is al

predicted question=[what did the Clinical_Attribute property_of Living_Beings?]  answer=[Bacterium]


In [26]:
keywords

array(['Bacterium causes Disorders',
       'Bacterium interacts_with Living_Beings', 'Bacterium isa Objects',
       'Bacterium issue_in Occupations',
       'Bacterium location_of Chemicals_&_Drugs',
       'Acquired_Abnormality affects Living_Beings',
       'Immunologic_Factor indicates Living_Beings',
       'Alga interacts_with Living_Beings',
       'Acquired_Abnormality location_of Living_Beings',
       'Acquired_Abnormality part_of Living_Beings',
       'Biologic_Function process_of Living_Beings',
       'Clinical_Attribute property_of Living_Beings'], dtype='<U74')

# BLEU score

In [41]:
import nltk
sum1=0
for i in range(len(actual)):
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([k1[i]], actual1[i])
    #rint(BLEUscore)
    sum1=sum1+BLEUscore
print(sum1/len(actual))

0.5887593466829465


In [37]:
actual1=[]
for i in actual:
    actual1.append(' '.join(i))
print(actual1)

['who is the current queen of australia', 'what to do at fairfield ca', 'what year did spudd webb won the dunk contest', 'who played maxie jones on general hospital', 'what are the major languages of italy', 'what language do they speak in indonesia', 'what is st francis patron saint of', 'what to do with kids in phx az', 'what kind of money does the uk use', 'who played atticus finch in the movie', 'who plays alan parrish in jumanji', 'who is hugh hefner dating now in', 'who does cristiano ronaldo play for now', 'which countries are part of the united kingdom', 'when did the celtics win championship', 'what jobs did albert einstein have', 'what are all the things thomas edison invented', 'where did matthew brady live', 'what will happen to greece when it defaults', 'who is the current president of colombia', 'who plays the brother in hannah montana', 'what did corey haim act in', 'who is the leader of japan right now', 'what countries does queen elizabeth ii reign', 'what are fun thin

In [38]:
k1=[]
for i in k:
    k1.append(' '.join(i))
print(k1)

['current queen australia', 'fairfield', 'year spudd webb dunk contest', 'played maxie jones general hospital', 'major languages italy', 'language speak indonesia', 'st francis patron saint', 'kids phx az', 'kind money uk use', 'played atticus finch movie', 'plays alan parrish jumanji', 'hugh hefner dating', 'cristiano ronaldo play', 'countries part united kingdom', 'celtics win championship', 'jobs albert einstein', 'things thomas edison invented', 'matthew brady live', 'happen greece defaults', 'current president colombia', 'plays hannah montana', 'corey haim act', 'leader japan right', 'countries queen elizabeth ii reign', 'fun things philadelphia', 'queen victoria queen', 'year lakers win first championship', 'traditions jamaica', 'last year nfl super bowl', 'discovered', 'government japan', 'first leader afl', 'beth tweddle compete', 'name wright brothers plane', 'mario lopez marry', 'official language china', 'team hank baskett play season', 'voice family guys meg griffin', 'owns

In [27]:
keywords

array(['Bacterium causes Disorders',
       'Bacterium interacts_with Living_Beings', 'Bacterium isa Objects',
       'Bacterium issue_in Occupations',
       'Bacterium location_of Chemicals_&_Drugs',
       'Acquired_Abnormality affects Living_Beings',
       'Immunologic_Factor indicates Living_Beings',
       'Alga interacts_with Living_Beings',
       'Acquired_Abnormality location_of Living_Beings',
       'Acquired_Abnormality part_of Living_Beings',
       'Biologic_Function process_of Living_Beings',
       'Clinical_Attribute property_of Living_Beings'], dtype='<U74')

In [39]:
predicted1=[]
for i in predicted:
    predicted1.append(' '.join(i))
print(predicted1)

['what is the current queen australia', 'what what is fairfield', 'what did did year spudd webb dunk contest', 'what is the played maxie jones general hospital', 'what is the major languages italy', 'what is the language speak indonesia', 'what is the st francis patron saint', 'what is the kids phx az', 'what did the kind money uk use', 'what is the played atticus finch movie', 'what is the plays alan parrish jumanji', 'what is the hugh hefner dating', 'what is the cristiano ronaldo play', 'what did the countries part united kingdom', 'what is the celtics win championship', 'what is the jobs albert einstein', 'what is the things thomas edison invented', 'what is the matthew brady live', 'what is the happen greece defaults', 'what is the current president colombia', 'what is the plays hannah montana', 'what is the corey haim act', 'what did the leader japan right', 'what did the countries queen elizabeth ii reign', 'what is the fun things philadelphia', 'what is the queen victoria queen